# Dogs vs Cats
## Kaggle Dataset의 전부를 이용한 개, 고양이 구분
### Dog Image: 12,500개, Cat Image: 12,500개, 총 25,000개
### 출처: [pontoregende GitHub](https://github.com/pontorezende/Dogs-vs-Cats-Redux-with-CNN)

In [1]:
from keras.preprocessing import image
from glob import glob
import cv2, os, random
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
%matplotlib inline

Using TensorFlow backend.


In [2]:
path='dataset/dogs-vs-cats/train'

In [8]:
## used for resize and in our model
ROW, COL = 96, 96

dogs, cats = [], []
y_dogs, y_cats = [], []

In [10]:
## Definition to load all our dog images
def load_dogs():
    print('Loading all dog images\n')
    dog_path = os.path.join(path, 'dog*')
    for dog_img in glob(dog_path):
        dog = cv2.imread(dog_img)
        dog = cv2.cvtColor(dog, cv2.COLOR_BGR2GRAY)
        dog = cv2.resize(dog, (ROW, COL))
        dog = image.img_to_array(dog)
        dogs.append(dog)
    print('All dog images loaded')

In [9]:
## Definition to load all our cat images
def load_cats():
    print('Loading all cat images\n')
    cat_path = os.path.join(path, 'cat*')
    for cat_img in glob(cat_path):
        cat = cv2.imread(cat_img)
        cat = cv2.cvtColor(cat, cv2.COLOR_BGR2GRAY)
        cat = cv2.resize(cat, (ROW, COL))
        cat = image.img_to_array(cat)
        cats.append(cat)
    print('All cat images loaded')

In [11]:
load_dogs()

Loading all dog images

All dog images loaded


In [12]:
load_cats()

Loading all cat images

All cat images loaded


In [13]:
classes = ['dog', 'cat']

In [14]:
## in case we want to see if our images was saved correctly in arrays we can use those codes
def show_dogs():
    plt.figure(figsize=(12,8))    
    for i in range(5):
        plt.subplot(1, 5, i+1)
        img = image.array_to_img(random.choice(dogs))
        plt.imshow(img)
        
        plt.axis('off')
        plt.title('Supposed to be a {}'.format(classes[0]))        
    plt.show()

In [15]:
def show_cats():
    plt.figure(figsize=(12,8))
    for i in range(5):
        plt.subplot(1, 5, i+1)
        img = image.array_to_img(random.choice(cats))
        plt.imshow(img)

        plt.axis('off')
        plt.title('Supposed to be a {}'.format(classes[1]))
    plt.show()

In [16]:
## just change the labels for 0 and  1
y_dogs = [1 for item in enumerate(dogs)]
y_cats = [0 for item in enumerate(cats)]

In [17]:
## converting everything to Numpy array to fit in our model
## them creating a X and target file like we used to see
## in Machine and Deep Learning models
dogs = np.asarray(dogs).astype('float32')
cats = np.asarray(cats).astype('float32')
y_dogs = np.asarray(y_dogs).astype('int32')
y_cats = np.asarray(y_cats).astype('int32')

In [18]:
## fit values between 0 and 1
dogs /= 255
cats /= 255

In [19]:
X = np.concatenate((dogs,cats), axis=0)
y = np.concatenate((y_dogs, y_cats), axis=0)

In [20]:
IMG_CHANNEL = 1
BATCH_SIZE = 128
N_EPOCH = 40
VERBOSE = 2
VALIDAION_SPLIT = .2
OPTIM = Adam()
N_CLASSES = len(classes)

In [21]:
## One-Hot Encoding
y = np_utils.to_categorical(y, N_CLASSES)
print('One-Hot Encoding done')

One-Hot Encoding done


In [22]:
## Here is our model as a CNN
model = Sequential([
    Conv2D(32, (3,3), padding='same', input_shape=(ROW, COL, IMG_CHANNEL), activation='relu'),
    Conv2D(32, (3,3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(.25),
    Conv2D(64, (3,3), padding='same', activation='relu'),
    Conv2D(64, (3,3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(.25),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(.5),
    Dense(N_CLASSES, activation='softmax')
])

print('The model was created by following config:')
model.summary()

W0811 21:58:56.034437  2988 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0811 21:58:56.037435  2988 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0811 21:58:56.047448  2988 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0811 21:58:56.104399  2988 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0811 21:58:56.108396  2988 deprecation_wrapper.py:119] From C:\ProgramData\Ana

The model was created by following config:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 96, 96, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 96, 96, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 48, 48, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 48, 48, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (Non

In [23]:
model.compile(loss='categorical_crossentropy', optimizer=OPTIM, metrics=['accuracy'])

W0811 22:02:48.080152  2988 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0811 22:02:48.094146  2988 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [25]:
## to save checkpoint to use latter
checkpoint = ModelCheckpoint('model_checkpoint/dogs_vs_cats_redux_checkpoint.h5')
model.fit(X, y, batch_size=BATCH_SIZE, epochs=N_EPOCH, validation_split=VALIDAION_SPLIT,
          verbose=VERBOSE, callbacks=[checkpoint])

Train on 20000 samples, validate on 5000 samples
Epoch 1/40
 - 1078s - loss: 0.5939 - acc: 0.6780 - val_loss: 0.8983 - val_acc: 0.4444
Epoch 2/40
 - 1098s - loss: 0.5316 - acc: 0.7340 - val_loss: 0.8564 - val_acc: 0.5462
Epoch 3/40
 - 1057s - loss: 0.4735 - acc: 0.7751 - val_loss: 0.7183 - val_acc: 0.6264
Epoch 4/40
 - 1037s - loss: 0.4212 - acc: 0.8062 - val_loss: 0.4502 - val_acc: 0.7956
Epoch 5/40
 - 1022s - loss: 0.3829 - acc: 0.8298 - val_loss: 0.5680 - val_acc: 0.7240
Epoch 6/40
 - 1019s - loss: 0.3359 - acc: 0.8511 - val_loss: 0.5727 - val_acc: 0.7272
Epoch 7/40
 - 1009s - loss: 0.2975 - acc: 0.8723 - val_loss: 0.7727 - val_acc: 0.6354
Epoch 8/40
 - 1023s - loss: 0.2476 - acc: 0.8973 - val_loss: 0.7022 - val_acc: 0.7106
Epoch 9/40
 - 1021s - loss: 0.2067 - acc: 0.9149 - val_loss: 0.8073 - val_acc: 0.6856
Epoch 10/40
 - 1037s - loss: 0.1611 - acc: 0.9360 - val_loss: 0.5314 - val_acc: 0.7858
Epoch 11/40
 - 1036s - loss: 0.1350 - acc: 0.9477 - val_loss: 0.7253 - val_acc: 0.7668
Epo

In [26]:
scores = model.evaluate(X, y, verbose=2)
print('MODEL ACCURACY\n{}: {}%'.format(model.metrics_names[1], scores[1]*100))

MODEL ACCURACY
acc: 95.304%


In [27]:
model_name = 'Dogs-and-Cats-12500-CNN'
## saving architecture
model_json = model.to_json()
open(model_name+'.json', 'w').write(model_json)
print('JSON saved')

## and the weights learned by our deep network on the training set
model.save(model_name+'.h5', overwrite=True)
print('.h5 saved')

model.save_weights(model_name+'_weights.h5', overwrite=True)
print('Weights saved in .h5 file')

JSON saved
.h5 saved
Weights saved in .h5 file
